# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import math
import requests
import yfinance as yf

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Life Sciences Tools & Services,"Santa Clara, California",05/06/00,1090872,1999
1,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",30/11/82,320193,1977
2,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",31/12/12,1551152,2013 (1888)
3,ABNB,Airbnb,Consumer Discretionary,"Hotels, Resorts & Cruise Lines","San Francisco, California",18/09/23,1559720,2008
4,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",04/03/57,1800,1888
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",01/11/11,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",06/10/97,1041061,1997
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",07/08/01,1136869,1927
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",23/12/19,877212,1969


## Making Our First API Call

Now it's time to structure our API calls to Alpha Vantage. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [54]:
symbol = 'AAPL'
stock = yf.Ticker(symbol)

yfinance.Ticker object <AAPL>

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [55]:
stock_price = stock.history(period="1d")['Close'].iloc[-1]
print(stock_price)

224.22999572753906


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [5]:
cols = [ 'Ticker', 'Stock Price', 'Number of Shares to Buy' ]
final_df = pd.DataFrame(columns = cols)
final_df

,Ticker,Stock Price,Number of Shares to Buy


In [6]:
new_row = pd.DataFrame([[symbol, stock_price, 'N/A']], columns=cols)


final_df = pd.concat([final_df, new_row])
final_df

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/3764206733.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row])


,Ticker,Stock Price,Number of Shares to Buy
0,AAPL,224.229996,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [34]:
final_df = pd.DataFrame(columns=cols)
for st in stocks['Symbol']:
    stock = yf.Ticker(st)
    stock_price = stock.history(period="1d")['Close'].iloc[-1]

    new_row = pd.DataFrame([[st, stock_price, 'N/A']], columns=cols)

    final_df = pd.concat([final_df, new_row], ignore_index=True)
final_df

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/829002857.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)


A
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMTM
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBY
BDX
BEN
BF-B
BG
BIIB
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CRWD
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DELL
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ERIE
ES
ESS
ETN
ETR
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOX
FOXA
FRT
FSLR
FTNT
FTV
GD
GDDY
GE
GEHC
GEN
GEV
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
I

,Ticker,Stock Price,Number of Shares to Buy
0,A,133.860001,N/A
1,AAPL,224.229996,N/A
2,ABBV,174.429993,N/A
3,ABNB,137.399994,N/A
4,ABT,116.800003,N/A
...,...,...,...
498,XYL,124.629997,N/A
499,YUM,136.619995,N/A
500,ZBH,109.459999,N/A
501,ZBRA,403.160004,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [21]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [36]:
symbol_grps = list(chunks(stocks['Symbol'], 100))
symbol_strings = []

for i in range(0, len(symbol_grps)):
    symbol_strings.append(' '.join(symbol_grps[i]))

final_df = pd.DataFrame(columns=cols)

for symbol_string in symbol_strings:
    batch_api_call = yf.download(symbol_string, period="1d")

    for symbol in symbol_string.split(' '):
        price = batch_api_call['Close'][symbol].iloc[-1]
        new_row = pd.DataFrame([[symbol, batch_api_call['Close'][symbol].iloc[-1], 'N/A']], columns=cols)

        final_df = pd.concat([final_df, new_row], ignore_index=True)

final_df

[*********************100%***********************]  100 of 100 completed
/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/57352750.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, new_row], ignore_index=True)
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  100 of 100 completed
[*********************100%***********************]  3 of 3 completed


,Ticker,Stock Price,Number of Shares to Buy
0,A,133.860001,N/A
1,AAPL,224.229996,N/A
2,ABBV,174.429993,N/A
3,ABNB,137.399994,N/A
4,ABT,116.800003,N/A
...,...,...,...
498,XYL,124.629997,N/A
499,YUM,136.619995,N/A
500,ZBH,109.459999,N/A
501,ZBRA,403.160004,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [49]:
portfolio_size = input('Enter the value of your portfolio: ')

try:
    val = float(portfolio_size)
except ValueError:
    print("Please enter an integer")
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [52]:
position_size = val/len(final_df.index)

for i in range(0, len(final_df.index)):
    price = final_df.loc[i, 'Stock Price']
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / price)

final_df

,Ticker,Stock Price,Number of Shares to Buy
0,A,133.860001,14
1,AAPL,224.229996,8
2,ABBV,174.429993,11
3,ABNB,137.399994,14
4,ABT,116.800003,17
...,...,...,...
498,XYL,124.629997,15
499,YUM,136.619995,14
500,ZBH,109.459999,18
501,ZBRA,403.160004,4


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [93]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

/var/folders/hd/h4ty4f0j2tgfss4vb7rrr93w0000gn/T/ipykernel_24002/3571317376.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_df.to_excel(writer, 'Recommended Trades', index=False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [94]:
background_color = '#0a0a23'
font_color = 'ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [95]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.close()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [97]:
col_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format]
}

for col in col_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 18, col_formats[col][1])
    writer.sheets['Recommended Trades'].write(f'{col}1:{col}1', col_formats[col][0], col_formats[col][1])


## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.close()